This comprehensive notebook teaches you **Natural Language Processing fundamentals** through a hands-on **Sentiment Analysis Project**. You'll learn both the theory and practical implementation of core NLP concepts.


## 1. Introduction to NLP

### What is Natural Language Processing?

**Definition**: Natural Language Processing is a branch of artificial intelligence that focuses on enabling computers to understand, interpret, analyze, and generate human language in a meaningful and useful way.

### Two Main Components

#### a) Natural Language Understanding (NLU)
- **Goal**: Make computers understand human language
- **Tasks**: Reading comprehension, question answering, named entity recognition
- **Challenge**: Ambiguity, context, idioms, sarcasm

#### b) Natural Language Generation (NLG)
- **Goal**: Enable computers to generate human-like text
- **Tasks**: Text summarization, machine translation, chatbot responses
- **Challenge**: Grammatical correctness, coherence, relevance

### Real-World Applications

| Application | Use Case | Example |
|---|---|---|
| **Sentiment Analysis** | Determine emotional tone of text | Reviews, social media monitoring |
| **Text Classification** | Categorize documents | Email spam detection, news categorization |
| **Named Entity Recognition** | Extract entities from text | Identifying person names, locations, organizations |
| **Machine Translation** | Translate between languages | Google Translate |
| **Question Answering** | Answer questions about documents | Chatbots, search engines |
| **Text Summarization** | Create concise summaries | News summaries, document abstracts |

---

## 2. Environment Setup

### Required Libraries

We'll use these popular Python NLP libraries:



In [34]:
!pip install nltk textblob spacy pandas numpy matplotlib scikit-learn
!python -m spacy download en_core_web_sm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 115.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


### Import All Libraries

In [35]:
# Data manipulation
import pandas as pd
import numpy as np

# NLP Libraries
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from textblob import TextBlob
import spacy

# Machine Learning
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

# Visualization
import matplotlib.pyplot as plt
from wordcloud import WordCloud

# Download required NLTK data
nltk.download('punkt')  # Tokenization
nltk.download('punkt_tab')
nltk.download('stopwords')  # Stop words
nltk.download('averaged_perceptron_tagger')  # POS tagging
nltk.download('wordnet')  # Lemmatization
nltk.download('maxent_ne_chunker')  # Named Entity Recognition
nltk.download('words')
nltk.download('averaged_perceptron_tagger_eng')

print(" All libraries imported successfully!")

 All libraries imported successfully!


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /roo

## 3. Core NLP Concepts

### 3.1 Tokenization

**Definition**: Breaking down text into smaller meaningful units called tokens (words, subwords, or characters)

**Why It Matters**: Most NLP algorithms operate on token-level basis, not raw text

**Types of Tokenization**:
- **Word Tokenization**: Split into words
- **Sentence Tokenization**: Split into sentences
- **Subword Tokenization**: Split into subwords (e.g., "unhappy" → "un" + "happy")


**Theory**: Tokenization is the foundation of NLP. Different languages require different tokenization strategies (e.g., Chinese has no word boundaries, requiring more sophisticated techniques).

---

**Example**

In [36]:
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk

text = "Natural Language Processing is amazing! It helps machines understand text."

# Word Tokenization
words = word_tokenize(text)
print("Words:", words)
# Output: ['Natural', 'Language', 'Processing', 'is', 'amazing', '!', 'It', ...]

# Sentence Tokenization
sentences = sent_tokenize(text)
print("Sentences:", sentences)
# Output: ['Natural Language Processing is amazing!', 'It helps machines understand text.']

Words: ['Natural', 'Language', 'Processing', 'is', 'amazing', '!', 'It', 'helps', 'machines', 'understand', 'text', '.']
Sentences: ['Natural Language Processing is amazing!', 'It helps machines understand text.']


### 3.2 Stop Words

**Definition**: Common words in a language that carry minimal meaningful information (the, is, and, or, etc.)

**Why Remove Them**: Stop words can obscure the main meaning and increase computational cost


**Caution**: Be careful when removing stop words from sentiment analysis (e.g., "not good" → "good" loses crucial information)

---

**Example**:

In [37]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))
print("Sample stop words:", list(stop_words)[:10])

text = "The quick brown fox jumps over the lazy dog"
tokens = word_tokenize(text.lower())
filtered_tokens = [token for token in tokens if token not in stop_words]

print("Original:", tokens)
print("Filtered:", filtered_tokens)
# Output - Filtered: ['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']

Sample stop words: ["he'll", 'out', 'each', 'from', 'can', 'further', "needn't", "i'm", "mustn't", 'over']
Original: ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
Filtered: ['quick', 'brown', 'fox', 'jumps', 'lazy', 'dog']


### 3.3 Lowercasing & Cleaning

**Definition**: Normalize text by converting to lowercase and removing special characters

**Why It Matters**:
- "Apple", "apple", and "APPLE" should be treated as the same token
- Numbers and special characters often don't add semantic value

**Example**:


In [38]:
import re

def clean_text(text):
    # Convert to lowercase
    text = text.lower()

    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)

    # Remove special characters and digits
    text = re.sub(r'[^a-zA-Z\s]', '', text)

    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text).strip()

    return text

raw_text = "Check this out!!! Visit https://example.com #NLP @Python123"
cleaned = clean_text(raw_text)
print("Original:", raw_text)
print("Cleaned:", cleaned)
# Output - Cleaned: "check this out visit example nlp python"

Original: Check this out!!! Visit https://example.com #NLP @Python123
Cleaned: check this out visit nlp python


## 4. Text Preprocessing Pipeline

**Definition**: A systematic approach to prepare raw text for analysis

**Steps in Order**:
1. Remove HTML/URLs
2. Convert to lowercase
3. Remove special characters
4. Tokenization
5. Remove stop words
6. Lemmatization/Stemming

**Example**:

In [39]:
def preprocess_text(text):
    """Complete text preprocessing pipeline"""
    # 1. Remove URLs and special patterns
    text = re.sub(r'http\S+|www\S+|@\S+|#\S+', '', text)

    # 2. Lowercase
    text = text.lower()

    # 3. Remove special characters (keep alphanumeric and spaces)
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # 4. Tokenize
    tokens = word_tokenize(text)

    # 5. Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words]

    # 6. Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    return ' '.join(tokens)

# Test
sample = "I'm absolutely LOVING this amazing NLP course! Check it out: https://akhil.com #ML"
result = preprocess_text(sample)
print("Input:", sample)
print("Output:", result)


Input: I'm absolutely LOVING this amazing NLP course! Check it out: https://akhil.com #ML
Output: im absolutely loving amazing nlp course check


## 5. Tokenization In Detail

### Different Tokenization Methods

In [40]:
from nltk.tokenize import word_tokenize, sent_tokenize
import spacy

text = "Dr. Smith works at Google Inc. He loves NLP!"

# Method 1: NLTK Word Tokenization
nltk_tokens = word_tokenize(text)
print("NLTK:", nltk_tokens)

# Method 2: NLTK Sentence Tokenization
nltk_sents = sent_tokenize(text)
print("Sentences:", nltk_sents)

# Method 3: spaCy Tokenization (More sophisticated)
nlp = spacy.load('en_core_web_sm')
doc = nlp(text)
spacy_tokens = [token.text for token in doc]
print("spaCy:", spacy_tokens)


NLTK: ['Dr.', 'Smith', 'works', 'at', 'Google', 'Inc', '.', 'He', 'loves', 'NLP', '!']
Sentences: ['Dr. Smith works at Google Inc.', 'He loves NLP!']
spaCy: ['Dr.', 'Smith', 'works', 'at', 'Google', 'Inc.', 'He', 'loves', 'NLP', '!']


### spaCy advantages:
- Handles abbreviations better ("Dr." is one token, not two)
- Provides additional linguistic information
- More accurate for complex text

## 6. Normalization & Text Cleaning

### Techniques Explained

**1. Case Normalization**



In [41]:
text = "Machine Learning and MACHINE learning"
normalized = text.lower()
print(normalized)

machine learning and machine learning



**2. Punctuation Removal**

In [42]:
import string

text = "Hello, world! How are you?"
no_punct = text.translate(str.maketrans('', '', string.punctuation))
print(no_punct)

Hello world How are you




**3. Number Handling**

In [43]:
# Option A: Remove all numbers
text = "I have 2 cats and 3 dogs"
no_nums = re.sub(r'\d+', '', text)
print(no_nums)

# Option B: Replace with placeholder
text_placeholder = re.sub(r'\d+', '[NUM]', text)
print(text_placeholder)


I have  cats and  dogs
I have [NUM] cats and [NUM] dogs


**4. Whitespace Normalization**

In [44]:
text = "Hello    world    from    NLP"
normalized = ' '.join(text.split())
print(normalized)

Hello world from NLP


## 7. Lemmatization & Stemming

### Lemmatization vs Stemming

| Aspect | Lemmatization | Stemming |
|--------|---------------|----------|
| **Method** | Uses dictionary & grammar rules | Rule-based heuristic |
| **Output** | Actual dictionary word | Word root (may not be real word) |
| **Accuracy** | Higher | Lower |
| **Speed** | Slower | Faster |
| **Use Case** | Precision needed | Speed priority |

**Lemmatization Example**:


In [45]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

words = ["running", "ran", "runs", "runner", "cars", "was", "being"]

for word in words:
    lemma = lemmatizer.lemmatize(word)
    print(f"{word:15} → {lemma}")

running         → running
ran             → ran
runs            → run
runner          → runner
cars            → car
was             → wa
being           → being


**Solution - POS Tagging Help**:

In [51]:
from nltk import pos_tag, word_tokenize

text = "The dogs were running in the park"
tokens = word_tokenize(text)
pos_tags = pos_tag(tokens)
print(pos_tags)

lemmatizer = WordNetLemmatizer()
lemmatized = []
for word, pos in pos_tags:
    if pos.startswith('V'):  # Verb
        lemma = lemmatizer.lemmatize(word, pos='v')
    else:
        lemma = lemmatizer.lemmatize(word)
    lemmatized.append(lemma)

print(' '.join(lemmatized))

[('The', 'DT'), ('dogs', 'NNS'), ('were', 'VBD'), ('running', 'VBG'), ('in', 'IN'), ('the', 'DT'), ('park', 'NN')]
The dog be run in the park


**Stemming Example**:

In [52]:
from nltk.stem import PorterStemmer

stemmer = PorterStemmer()

words = ["running", "ran", "runs", "runner", "connection", "connecting"]

for word in words:
    stem = stemmer.stem(word)
    print(f"{word:15} → {stem}")

running         → run
ran             → ran
runs            → run
runner          → runner
connection      → connect
connecting      → connect


## 8. Sentiment Analysis Project {#project}

### Project Overview

**Objective**: Build a sentiment analysis model to classify text as Positive, Negative, or Neutral

**Dataset**: Reviews from various sources

**Skills Practiced**:
- Text preprocessing
- Feature extraction
- Machine learning classification
- Model evaluation

### 8.1 Create Sample Dataset

In [50]:
# Create sample dataset of reviews
reviews = [
    "I absolutely love this product! Best purchase ever!",
    "Terrible quality. Completely disappointed with my order.",
    "It's okay. Nothing special but does the job.",
    "Amazing customer service and fast delivery!",
    "Waste of money. Product broke after one day.",
    "Not bad. I'd recommend it to friends.",
    "Fantastic experience! Will definitely buy again!",
    "Awful. Poor quality and worse than expected.",
    "Good value for money. Pretty satisfied.",
    "Horrible experience. Never buying again."
]

sentiments = [
    "positive",
    "negative",
    "neutral",
    "positive",
    "negative",
    "neutral",
    "positive",
    "negative",
    "positive",
    "negative"
]

# Create DataFrame
df = pd.DataFrame({
    'review': reviews,
    'sentiment': sentiments
})

print(df)
print(f"\nDataset shape: {df.shape}")
print(f"Sentiment distribution:\n{df['sentiment'].value_counts()}")

                                              review sentiment
0  I absolutely love this product! Best purchase ...  positive
1  Terrible quality. Completely disappointed with...  negative
2       It's okay. Nothing special but does the job.   neutral
3        Amazing customer service and fast delivery!  positive
4       Waste of money. Product broke after one day.  negative
5              Not bad. I'd recommend it to friends.   neutral
6   Fantastic experience! Will definitely buy again!  positive
7       Awful. Poor quality and worse than expected.  negative
8            Good value for money. Pretty satisfied.  positive
9           Horrible experience. Never buying again.  negative

Dataset shape: (10, 2)
Sentiment distribution:
sentiment
positive    4
negative    4
neutral     2
Name: count, dtype: int64


### 8.2 Text Preprocessing for Reviews

In [53]:
def preprocess_review(text):
    """Preprocess review text"""
    # Lowercase
    text = text.lower()

    # Remove special characters
    text = re.sub(r'[^a-z0-9\s]', '', text)

    # Tokenize
    tokens = word_tokenize(text)

    # Remove stop words
    stop_words = set(stopwords.words('english'))
    tokens = [t for t in tokens if t not in stop_words and len(t) > 2]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]

    return ' '.join(tokens)

# Apply preprocessing
df['processed_review'] = df['review'].apply(preprocess_review)

print("\nBefore Processing:")
print(df['review'].iloc[0])
print("\nAfter Processing:")
print(df['processed_review'].iloc[0])


Before Processing:
I absolutely love this product! Best purchase ever!

After Processing:
absolutely love product best purchase ever


### 8.3 Feature Extraction

**What is Feature Extraction?**
Text is unstructured. ML models need numbers. Feature extraction converts text into numerical features.

**Method 1: Bag of Words (BoW)**

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

# Create BoW vectors
bow_vectorizer = CountVectorizer(max_features=20)
bow_features = bow_vectorizer.fit_transform(df['processed_review'])

print("Vocabulary:", bow_vectorizer.get_feature_names_out())
print("Feature shape:", bow_features.shape)
print("Sample features (first review):")
print(bow_features[0].toarray())


Vocabulary: ['absolutely' 'amazing' 'awful' 'bad' 'best' 'broke' 'completely'
 'customer' 'day' 'definitely' 'delivery' 'disappointed' 'ever' 'expected'
 'experience' 'fantastic' 'fast' 'money' 'product' 'quality']
Feature shape: (10, 20)
Sample features (first review):
[[1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0]]



**Explanation**: Bag of Words counts occurrences of each word. Output is a matrix where each column represents a word, and each row represents a document.

**Method 2: TF-IDF (Term Frequency-Inverse Document Frequency)**

In [55]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Create TF-IDF vectors
tfidf_vectorizer = TfidfVectorizer(max_features=20)
tfidf_features = tfidf_vectorizer.fit_transform(df['processed_review'])

print("Feature shape:", tfidf_features.shape)
print("Sample TF-IDF weights (first review):")
print(tfidf_features[0].toarray())


Feature shape: (10, 20)
Sample TF-IDF weights (first review):
[[0.5182909  0.         0.         0.         0.5182909  0.
  0.         0.         0.         0.         0.         0.
  0.5182909  0.         0.         0.         0.         0.
  0.44059462 0.        ]]


**Why TF-IDF Better Than BoW**?
- **TF**: How often a word appears in a document
- **IDF**: How unique a word is across all documents
- **Result**: Common words get lower weights, rare meaningful words get higher weights

### 8.4 Build Sentiment Classifier


In [56]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# For larger datasets, use train-test split
# For now, we'll train on full dataset for demonstration

# Create features using TF-IDF
tfidf = TfidfVectorizer(max_features=30, stop_words='english')
X = tfidf.fit_transform(df['processed_review'])
y = df['sentiment']

# Create and train classifier
classifier = MultinomialNB()
classifier.fit(X, y)

# Predictions
y_pred = classifier.predict(X)

# Evaluation Metrics
print("📊 MODEL PERFORMANCE")
print("="*50)
print(f"Accuracy:  {accuracy_score(y, y_pred):.4f}")
print(f"Precision: {precision_score(y, y_pred, average='weighted', zero_division=0):.4f}")
print(f"Recall:    {recall_score(y, y_pred, average='weighted'):.4f}")
print(f"F1-Score:  {f1_score(y, y_pred, average='weighted'):.4f}")

# Confusion Matrix
cm = confusion_matrix(y, y_pred)
print("\nConfusion Matrix:")
print(cm)

📊 MODEL PERFORMANCE
Accuracy:  1.0000
Precision: 1.0000
Recall:    1.0000
F1-Score:  1.0000

Confusion Matrix:
[[4 0 0]
 [0 2 0]
 [0 0 4]]


**Explanation of Metrics**:
- **Accuracy**: % of correct predictions overall
- **Precision**: Of positive predictions, how many were correct?
- **Recall**: Of actual positives, how many did we catch?
- **F1-Score**: Harmonic mean of precision and recall

### 8.5 Test on New Reviews

In [57]:
def predict_sentiment(text):
    """Predict sentiment of new text"""
    # Preprocess
    processed = preprocess_review(text)

    # Vectorize
    features = tfidf.transform([processed])

    # Predict
    prediction = classifier.predict(features)[0]
    confidence = max(classifier.predict_proba(features)[0])

    return prediction, confidence

# Test on new reviews
test_reviews = [
    "This product is fantastic! Love it!",
    "Absolutely horrible. Do not buy.",
    "It's alright, nothing special."
]

print("\n TESTING ON NEW REVIEWS")
print("="*50)
for review in test_reviews:
    sentiment, confidence = predict_sentiment(review)
    print(f"Review: {review}")
    print(f"Sentiment: {sentiment} (Confidence: {confidence:.2%})")
    print()


 TESTING ON NEW REVIEWS
Review: This product is fantastic! Love it!
Sentiment: positive (Confidence: 51.97%)

Review: Absolutely horrible. Do not buy.
Sentiment: positive (Confidence: 45.36%)

Review: It's alright, nothing special.
Sentiment: negative (Confidence: 40.00%)



## 9. Advanced Concepts

### 9.1 Named Entity Recognition (NER)

**Definition**: Identifying and classifying named entities in text (persons, organizations, locations, etc.)


In [58]:
import spacy

nlp = spacy.load('en_core_web_sm')

text = "Apple CEO Tim Cook announced new products in San Francisco on January 12, 2026."
doc = nlp(text)

print("Named Entities:")
print("-" * 50)
for ent in doc.ents:
    print(f"{ent.text:20} → {ent.label_:15} ({spacy.explain(ent.label_)})")

Named Entities:
--------------------------------------------------
Apple                → ORG             (Companies, agencies, institutions, etc.)
Tim Cook             → PERSON          (People, including fictional)
San Francisco        → GPE             (Countries, cities, states)
January 12, 2026     → DATE            (Absolute or relative dates or periods)


### 9.2 Part-of-Speech (POS) Tagging

**Definition**: Assigning grammatical roles to words (noun, verb, adjective, etc.)


In [60]:
text = "The cat quickly jumped over the fence."
doc = nlp(text)

print("POS Tagging:")
print("-" * 50)
for token in doc:
    print(f"{token.text:15} → {token.pos_:10} ({token.tag_})")

POS Tagging:
--------------------------------------------------
The             → DET        (DT)
cat             → NOUN       (NN)
quickly         → ADV        (RB)
jumped          → VERB       (VBD)
over            → ADP        (IN)
the             → DET        (DT)
fence           → NOUN       (NN)
.               → PUNCT      (.)


### 9.3 Dependency Parsing

**Definition**: Understanding grammatical relationships between words

In [61]:
text = "The quick brown fox jumps over the lazy dog."
doc = nlp(text)

print("Dependency Parsing:")
print("-" * 50)
for token in doc:
    print(f"{token.text:15} → {token.dep_:10} (head: {token.head.text})")

# Visualization
from spacy import displacy
displacy.render(doc, style="dep")

Dependency Parsing:
--------------------------------------------------
The             → det        (head: fox)
quick           → amod       (head: fox)
brown           → amod       (head: fox)
fox             → nsubj      (head: jumps)
jumps           → ROOT       (head: jumps)
over            → prep       (head: jumps)
the             → det        (head: dog)
lazy            → amod       (head: dog)
dog             → pobj       (head: over)
.               → punct      (head: jumps)


### 9.4 Word Embeddings Introduction

**Definition**: Representing words as dense vectors in continuous space where similar words are close together

**Concept**: Instead of one-hot encoding (sparse), use embeddings (dense, semantic meaning)


In [62]:
# Using spaCy word vectors
text = "The quick brown fox"
doc = nlp(text)

print("Word Vectors (first 5 dimensions):")
print("-" * 50)
for token in doc:
    print(f"{token.text:15} → {token.vector[:5]}")

# Similarity computation
word1 = nlp("dog")
word2 = nlp("cat")
word3 = nlp("apple")

print(f"\nSimilarity Scores:")
print(f"dog - cat: {word1.similarity(word2):.4f} (similar)")
print(f"dog - apple: {word1.similarity(word3):.4f} (different)")

Word Vectors (first 5 dimensions):
--------------------------------------------------
The             → [ 0.44080257  0.01434215  0.23331341  0.6461768  -0.27183467]
quick           → [ 1.072269   -0.88797593  0.9165765   0.5815992  -0.8369408 ]
brown           → [ 0.5068223  -0.86238426  0.88640505  0.15510865 -0.683393  ]
fox             → [ 0.04414169 -0.54116184 -0.319565    0.6428287   0.30003244]

Similarity Scores:
dog - cat: 0.7423 (similar)
dog - apple: 0.6848 (different)


/tmp/ipython-input-2404736424.py:16: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors, or use one of the larger models instead if available.
  print(f"dog - cat: {word1.similarity(word2):.4f} (similar)")
/tmp/ipython-input-2404736424.py:17: UserWarning: [W007] The model you're using has no word vectors loaded, so the result of the Doc.similarity method will be based on the tagger, parser and NER, which may not give useful similarity judgements. This may happen if you're using one of the small models, e.g. `en_core_web_sm`, which don't ship with word vectors and only use context-sensitive tensors. You can always add your own word vectors,

## 10. Complete Project Workflow


In [63]:
# COMPLETE SENTIMENT ANALYSIS PIPELINE

import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import re

# Step 1: Prepare Data
reviews_data = {
    'text': [
        "This is the best product I've ever used!",
        "Terrible quality, completely disappointed.",
        "Average product, nothing special.",
        "Absolutely amazing! Highly recommended!",
        "Waste of money. Broke after one week.",
        "Decent value for the price.",
    ],
    'label': ['positive', 'negative', 'neutral', 'positive', 'negative', 'neutral']
}

df = pd.DataFrame(reviews_data)

# Step 2: Create Pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(lowercase=True, stop_words='english', max_features=50)),
    ('classifier', MultinomialNB())
])

# Step 3: Train
pipeline.fit(df['text'], df['label'])

# Step 4: Predict
test_text = "This product is absolutely wonderful!"
prediction = pipeline.predict([test_text])[0]
confidence = max(pipeline.predict_proba([test_text])[0])

print(f"Text: {test_text}")
print(f"Sentiment: {prediction}")
print(f"Confidence: {confidence:.2%}")

Text: This product is absolutely wonderful!
Sentiment: positive
Confidence: 42.36%


## Key Takeaways

- **Tokenization** breaks text into meaningful units
- **Preprocessing** (cleaning, lemmatization) improves model performance
- **Feature Extraction** (BoW, TF-IDF) converts text to numbers
- **Stop Words** often removed to improve signal-to-noise ratio
- **Lemmatization** better than stemming for accuracy (but slower)
- **TF-IDF** better than BoW for representing important words
- **NER** identifies entities like people, places, organizations
- **Word embeddings** capture semantic meaning

---

## Practice Exercises

1. **Exercise 1**: Build a sentiment classifier on your own dataset
2. **Exercise 2**: Implement a spam detection system using NLP
3. **Exercise 3**: Create a text summarization tool
4. **Exercise 4**: Build a simple chatbot using NLP techniques
5. **Exercise 5**: Perform NER on news articles

---

## Further Learning Resources

- **Books**: "Natural Language Processing with Python" (NLTK Book)
- **Libraries**: spaCy, NLTK, TextBlob, Hugging Face Transformers
- **Advanced Topics**: BERT, GPT, Transformers, Transfer Learning
- **Datasets**: Movie Reviews, Restaurant Reviews, Tweet Sentiment datasets

---
